In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import tensorboard


In [ ]:
ndim = 2    # number of dimensions
T = 500     # maximum time steps := length of each trajectory
M = 250     # number of generated trajectories from observations of pi_explore


In [ ]:
d_min = 1   # min distance between each point
d_max = 100 # max distance between each point
trajectories = np.zeros(shape=(M, T, ndim), dtype=np.float)
for m in range(M):
    for t in range(1, T):
        trajectories[m, t] = trajectories[m, t-1] + np.random.randint(low=d_min, high=d_max, size=ndim, dtype='l')


In [ ]:
a_v = 2 # constant factor
a_x = 2 # constant factor
velocity_next = lambda velocity, acceleration  : velocity + a_v * acceleration
position_next = lambda position, velocity      : position + a_x * velocity


In [ ]:
a_max = d_max               # maximum acceleration
a_min = -a_max              # minimum acceleration
a = np.zeros(shape=ndim)    # current acceleration


In [ ]:
v_max = d_max                   # maximum velocity
v_min = -v_max                  # minimum velocity
s = np.zeros(shape=(2, ndim))   # current state
x = s[0,:]                      # current position
v = s[1,:]                      # current velocity



In [ ]:
# state must also include some observation about the next target(s)
pi_explore = lambda state : np.random.randint(low=a_min, high=a_max, size=ndim, dtype='l') # random policy
pi = keras.Sequential([
    keras.layers.Dense(32, input_shape=s.shape),
    keras.layers.Activation('relu'),
    keras.layers.Dense(a.size),
    keras.layers.Activation('tanh') # output: [-1: 1]
])


In [ ]:
f_s = keras.Sequential([
    keras.layers.Dense(32, input_shape=np.vstack([s, a]).shape),
    keras.layers.Activation('relu'),
    keras.layers.Dense(s.size),
    keras.layers.Activation('tanh') # output: [-1: 1]
])


In [ ]:
f_r = keras.Sequential([
    keras.layers.Dense(32, input_shape=np.vstack([s, a]).shape),
    keras.layers.Activation('relu'),
    keras.layers.Dense(1),
    keras.layers.Activation('tanh') # output: [-1: 1]
])


In [ ]:
num_episodes = 1    # := epochs
for _ in range(num_episodes):

    # for each trajectory (or perhaps create mini batches # => no minibatches, because trajectories may end at different times?)
    for trajectory in range(T):

        # reset:
        R = 0
        r = 0
        s = np.zeros(shape=(2, ndim))
        a = np.zeros(shape=ndim)

        # for each step:
        for t in range(T):

            # a = pi(s)
            # r = f_r(s, a)
            a = pi(s.reshape(-1))
            a = pi_explore(s.reshape(-1))
            r = f_r(np.vstack(s, a).reshape(-1))
            R += r